### Data Ingestion

In [1]:
### Document Data Structure
from langchain_core.documents import Document

In [3]:
doc  = Document(
    page_content="This is the content of the document, that I'll be using to create RAG",
    metadata={"source": "example.txt",
              "pages":1,
              "author":"Jignesh Patel",
              "date_created":"2026-02-04"
              }
)
doc

Document(metadata={'source': 'example.txt', 'pages': 1, 'author': 'Jignesh Patel', 'date_created': '2026-02-04'}, page_content="This is the content of the document, that I'll be using to create RAG")

In [4]:
## Create a txt file
import os
os.makedirs("../data/text_files", exist_ok=True)

In [5]:
sample_texts={
    "../data/text_files/python_intro.txt":"""Python Programming Introduction

Python is a high-level, interpreted programming language known for its simplicity and readability.
Created by Guido van Rossum and first released in 1991, Python has become one of the most popular
programming languages in the world.

Key Features:
- Easy to learn and use
- Extensive standard library
- Cross-platform compatibility
- Strong community support

Python is widely used in web development, data science, artificial intelligence, and automation.""",
    
    "../data/text_files/machine_learning.txt": """Machine Learning Basics

Machine learning is a subset of artificial intelligence that enables systems to learn and improve
from experience without being explicitly programmed. It focuses on developing computer programs
that can access data and use it to learn for themselves.

Types of Machine Learning:
1. Supervised Learning: Learning with labeled data
2. Unsupervised Learning: Finding patterns in unlabeled data
3. Reinforcement Learning: Learning through rewards and penalties

Applications include image recognition, speech processing, and recommendation systems
    
    
    """

}

for filepath, content in sample_texts.items():
    with open(filepath, "w", encoding="utf-8") as f:
        f.write(content)

print("Sample text files created")

Sample text files created


In [ ]:
### Langchain TextLoader
from langchain_community.document_loaders import TextLoader

loader=TextLoader("../data/text_files/python_intro.txt",encoding="utf-8")
document= loader.load()
print(document)

[Document(metadata={'source': '../data/text_files/python_intro.txt'}, page_content='Python Programming Introduction\n\nPython is a high-level, interpreted programming language known for its simplicity and readability.\nCreated by Guido van Rossum and first released in 1991, Python has become one of the most popular\nprogramming languages in the world.\n\nKey Features:\n- Easy to learn and use\n- Extensive standard library\n- Cross-platform compatibility\n- Strong community support\n\nPython is widely used in web development, data science, artificial intelligence, and automation.')]


In [15]:
### Directory Loader
from langchain_community.document_loaders import DirectoryLoader

dir_loader = DirectoryLoader(
    "../data/text_files",
    glob="*.txt",
    loader_cls=TextLoader,
    loader_kwargs={'encoding': "utf-8"},
)
documents = dir_loader.load()
print(documents)

[Document(metadata={'source': '..\\data\\text_files\\machine_learning.txt'}, page_content='Machine Learning Basics\n\nMachine learning is a subset of artificial intelligence that enables systems to learn and improve\nfrom experience without being explicitly programmed. It focuses on developing computer programs\nthat can access data and use it to learn for themselves.\n\nTypes of Machine Learning:\n1. Supervised Learning: Learning with labeled data\n2. Unsupervised Learning: Finding patterns in unlabeled data\n3. Reinforcement Learning: Learning through rewards and penalties\n\nApplications include image recognition, speech processing, and recommendation systems\n\n\n    '), Document(metadata={'source': '..\\data\\text_files\\python_intro.txt'}, page_content='Python Programming Introduction\n\nPython is a high-level, interpreted programming language known for its simplicity and readability.\nCreated by Guido van Rossum and first released in 1991, Python has become one of the most popul

In [16]:
import os
os.makedirs("../data/pdf_files", exist_ok=True)

In [1]:
import pathlib
import requests

# Folder (relative to your notebook)
pdf_dir = pathlib.Path("../data/pdf_files")
pdf_dir.mkdir(parents=True, exist_ok=True)

# Add/remove URLs as you like
pdfs = [
    ("https://dslinc.com/wp-content/uploads/2025/01/20105027_TUM_A400_en.pdf",
     "Franke_A400_User_Manual_EN.pdf"),

    ("https://elektro-serwis.com/wp-content/uploads/2020/03/WMF-1500S-User-Manual_EN.pdf",
     "WMF_1500S_User_Manual_EN.pdf"),

    ("https://www.simonelliusa.com/PDFs/Discontinued/Appia%20II/Appia%20II%202-3%20gr%20manual.pdf",
     "Nuova_Simonelli_Appia_II_2-3Group_Manual.pdf"),

    ("https://www.ranciliogroupna.com/wp-content/uploads/2021/05/MAN_Classe5-2020-10.pdf",
     "Rancilio_Classe_5_User_Manual.pdf"),
]

for url, filename in pdfs:
    out_path = pdf_dir / filename
    print("Downloading:", filename)

    r = requests.get(url, timeout=60, headers={"User-Agent": "Mozilla/5.0"})
    r.raise_for_status()

    out_path.write_bytes(r.content)

print("\nSaved to:", pdf_dir.resolve())
print("Files:")
for p in sorted(pdf_dir.glob("*.pdf")):
    print("-", p.name, f"({p.stat().st_size/1024/1024:.2f} MB)")


Downloading: Franke_A400_User_Manual_EN.pdf
Downloading: WMF_1500S_User_Manual_EN.pdf
Downloading: Nuova_Simonelli_Appia_II_2-3Group_Manual.pdf
Downloading: Rancilio_Classe_5_User_Manual.pdf

Saved to: C:\dev\traditional-rag-pipeline\data\pdf_files
Files:
- Franke_A400_User_Manual_EN.pdf (4.59 MB)
- Nuova_Simonelli_Appia_II_2-3Group_Manual.pdf (6.92 MB)
- Rancilio_Classe_5_User_Manual.pdf (2.89 MB)
- WMF_1500S_User_Manual_EN.pdf (1.59 MB)


In [4]:
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader,PyMuPDFLoader

dir_pdf_loader = DirectoryLoader(
    "../data/pdf_files",
    glob="*.pdf",
    loader_cls=PyMuPDFLoader,
    show_progress=False
)
pdf_documents = dir_pdf_loader.load()
pdf_documents

[Document(metadata={'producer': 'GPL Ghostscript 9.55.0', 'creator': 'SCHEMA ST4', 'creationdate': "D:20250507163426Z00'00'", 'source': '..\\data\\pdf_files\\Franke_A400_User_Manual_EN.pdf', 'file_path': '..\\data\\pdf_files\\Franke_A400_User_Manual_EN.pdf', 'total_pages': 84, 'format': 'PDF 1.4', 'title': '20105027_TUM_A400_en', 'author': '', 'subject': '', 'keywords': '', 'moddate': "D:20250507163426Z00'00'", 'trapped': '', 'modDate': "D:20250507163426Z00'00'", 'creationDate': "D:20250507163426Z00'00'", 'page': 0}, page_content='USER MANUAL\nA400\nFCS4060\nen\nRead the user manual before working on the machine.\nKeep the manual with the machine and pass the manual on to the\nnext operator if the machine is sold or transferred to a third party.'),
 Document(metadata={'producer': 'GPL Ghostscript 9.55.0', 'creator': 'SCHEMA ST4', 'creationdate': "D:20250507163426Z00'00'", 'source': '..\\data\\pdf_files\\Franke_A400_User_Manual_EN.pdf', 'file_path': '..\\data\\pdf_files\\Franke_A400_Use

In [5]:
type(pdf_documents[0])

langchain_core.documents.base.Document